In [ ]:
import requests
import pandas as pd
import time
import os

regioes = { # Regiões definidas na documentação do IBGE
    "Norte": 1,
    "Nordeste": 2,
    "Sudeste": 3,
    "Sul": 4,
    "Centro-Oeste": 5
}

dados = []
tabela = 1757 # Numero da tabela
variavel = 410 # Numero de empresas ativas 
periodo = "2007-2020" # Anos que desejamos buscar

for regiao_nome, regiao_codigo in regioes.items():
    url = f"https://apisidra.ibge.gov.br/values/t/{tabela}/n2/{regiao_codigo}/p/{periodo}/v/{variavel}?formato=json" #URL de consulta da API
    
    try:
        response = requests.get(url, timeout=30)
        if response.status_code == 200:
            data = response.json()[1:]
            for row in data:
                ano = int(row['D2N'])
                valor_str = row['V'].replace(".", "")
                valor = int(valor_str) if valor_str not in ['...', '-', ''] else None
                dados.append({'Ano': ano, 'Região': regiao_nome, 'Empresas_Ativas': valor})
        else:
            print(f"Erro {response.status_code} na região {regiao_nome}")
    except Exception as e:
        print(f"Erro de conexão na região {regiao_nome}: {e}")
    
    time.sleep(0.3)


# Criacao da pasta de 'data' caso não exista
parent_dir = os.path.dirname(os.getcwd())
output_dir = os.path.join(parent_dir, 'data')

# Salvamento do arquivo
os.makedirs(output_dir, exist_ok=True)

df = pd.DataFrame(dados)
output_path = os.path.join(output_dir, '0 - empresas_ativas_por_regiao.csv')
df.to_csv(output_path, index=False)

print(df.head())

    Ano Região  Empresas_Ativas
0  2007  Norte             2209
1  2008  Norte             2272
2  2009  Norte             2574
3  2010  Norte             3129
4  2011  Norte             3489
